In [ ]:
%tensorflow_version 1.15
!pip install tensorflow-gpu==1.15.0
!pip install keras_bert==0.84.0
!pip install keras==2.3.1
!pip install git+https://www.github.com/keras-team/keras-contrib.git


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
     |████████████████████████████████| 411.5MB 43kB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=5b3a9b5d6714a2ce410ff0591c46e93e809b5bf14fab6eb19d29007fce507b66
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none-any.whl size=36140 sha256=d86fa09ae8084bac8dc47f279314aaea328e34fd02ecbeb9c397a5707cea1d29
  Stored in directory: /root/.cache/pip/wheels/1f/59/04/12e95257aebfd27f7edaaf65ab7dd57b5f6cadfb183f1a4ccd
  Created wheel for keras-transformer: filename=keras_transformer-0.38.0-cp36-none-any.whl size=12942 sha256=6b618e3a71d3c2d398c2fd27a691ac3

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Lambda, Bidirectional, LSTM, Dense
from keras_bert import load_trained_model_from_checkpoint
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_accuracy
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from datetime import datetime
import keras
from collections import Counter
import keras.callbacks
import re
import codecs
import time
import pandas as pd
import os
import random

Using TensorFlow backend.


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
0.16218698900001982
GPU (s):
0.16630181799999377
GPU speedup over CPU: 0x


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/chinese_L-12_H-768_A-12/
%ls 

Mounted at /gdrive
/gdrive/My Drive/chinese_L-12_H-768_A-12
bert_config.json                     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [ ]:
# load and clean data
data = pd.read_excel(r'/gdrive/My Drive/law/判決書.xlsx')
data['content'] = data['content'].str.replace('[ 　]', '')
data['content'] = data['content'].str.replace(r'\xa0', '')
data['content'] = data['content'].str.replace(r'\n\n', '\n')
data['content'] = data['content'].str.replace(r'\n[0-9]{1,}\r', '')
data['content'] = data['content'].str.replace(r'書記官.*', '')
data['content'] = data['content'].str.replace(r'如不服本判決.*', '')
data['content'] = data['content'].str.replace(r'[┌├└┐┤┘┬┴│┼─]', '')
# 因為之前訓練model會訓練出位置的訊息，因此決定不取固定位置的文本
# 讓文本取的長度都是510，但位置不同
def content_slice(contents):
    contents_slice = []
    for content in contents:
        a = random.randint(50, len(content))
        contents_slice.append(content[a:a+510])

    return contents_slice
data['content'] = content_slice(data['content'])

test = data.iloc[25000:]
data = data.iloc[:25000, ]
# data.iloc[0]
# data['content'].iloc[0]
# data['content'].iloc[-1]
# data['url'].iloc[0]
data.shape

(25000, 3)

In [ ]:
# given BERT data path
bert_dir = r'/gdrive/My Drive/chinese_L-12_H-768_A-12/'
config_path = os.path.join(bert_dir, 'bert_config.json')
checkpoint_path = os.path.join(bert_dir, 'bert_model.ckpt')
dict_path = os.path.join(bert_dir, 'vocab.txt')

# given globel para
maxlen = 512
batch_size = 128
epochs = 2
input_shape = (maxlen, )


In [ ]:
def create_tokenizer(dict_path):
    '建BERT的字典'
    token_dict = {}
    with codecs.open(dict_path, 'r', 'utf8') as reader:
        for line in reader:
            token = line.strip()
            token_dict[token] = len(token_dict)
            
    return token_dict

# create BERT tokenizer
token_dict = create_tokenizer(dict_path)
tokenizer = Tokenizer(token_dict)

In [ ]:
def encoded(tokenizer, data, maxlen):
    '把新聞轉成 model input, 三個向量。只取後512個字'
    tokens, segments, masks = [], [], []
    for content in data['content']:
        # break
        if len(content) > 510:
            content = content[-510:]
        token, segment = tokenizer.encode(
            first=content, max_len=maxlen
            )
        mask = [1] * len(token)
        tokens.append(token)
        segments.append(segment)
        masks.append(mask)
            
    return tokens, segments, masks

# 產生input
token_input, segment_input, mask_input = encoded(tokenizer, data, maxlen)

In [ ]:
def create_label_crf(tokenizer, token_input, judge, maxlen):
    '把法官名稱轉成model label，兩個向量，分別記錄名字的開始、結束位置'
    # 產生空list
    # [[0] * maxlen]*len(token_input) 類似這種寫法，是淺複製
    # 會成為len(token_input)個一樣的list
    # start_label = list(temp) * len(token_input)也是淺複製
    temp = [0] * maxlen
    label = []
    for i in range(len(token_input)):
        label.append(list(temp))
    
    # 利用已知的法官名稱，標記出兩個one-hot的vector
    # start, end分別記錄一個法官名字的開始與結尾
    for a, (tokens, ju) in enumerate(zip(token_input, judge)): # 每個文本
        for name in ju: # 每個人名
            name_tokens = tokenizer.encode(name)[0][1:-1] # 名字轉換成tokenid
            # 對整個list遍歷，找到len(name_tokens)裡面的組合
            for tt, token in enumerate(tokens):
                # 找到開頭一樣
                if token == name_tokens[0]:
                    # 且後面幾個也都一樣
                    if tokens[tt:(tt+len(name_tokens))] == name_tokens:
                        # 紀錄下來
                        for j in range(tt, (tt+len(name_tokens))):
                            label[a][j] = 2
                        label[a][tt] = 1
                        
    return label

# 提取法官當作label
judge = data['content'].str[-510:].str.findall(r'\n法官(.{2,5})\r')
# 產生label
label = create_label_crf(tokenizer, token_input, judge, maxlen)

In [ ]:
# 人名次數分佈
judge.map(len).value_counts()

0    20290
1     2704
2     1588
4      403
3       15
Name: content, dtype: int64

In [ ]:
a = 0
idx = label[a].index(1)
st, ed = idx-10, idx+40
print(token_input[a][st:ed])
print(segment_input[a][st:ed])
print(mask_input[a][st:ed])
print(label[a][st:ed])


[2431, 2182, 1161, 7269, 3791, 2135, 1425, 4247, 3791, 2135, 2528, 3208, 7093, 3791, 2135, 6972, 989, 1923, 3791, 2135, 3360, 2617, 2255, 3791, 2135, 6258, 7093, 1313, 3315, 816, 3633, 3315, 6349, 3209, 5645, 1333, 3315, 4192, 4530, 704, 5836, 3696, 1751, 8965, 2399, 8108, 3299, 8143, 3189, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1, 2, 2, 0, 0, 1, 2, 2, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# tokenizer.encode('例稿名稱：臺灣屏東地方法院公示', max_len=512)
print(type(token_input), type(segment_input), type(mask_input), type(label))

<class 'list'> <class 'list'> <class 'list'> <class 'list'>


In [ ]:
def bert_BiLSTM_CRF_model():
    'NER模型'
    ner_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True, seq_len=maxlen)
    bert_output = ner_model.layers[-9].output
    X = Lambda(lambda x: x[:, 0: input_shape[0]])(bert_output)
    X = Bidirectional(LSTM(128, return_sequences=True))(X)

    output = CRF(3, sparse_target = True)(X)
    ner_model = Model(ner_model.input, output)
    
    for layer in ner_model.layers:
        layer.trainable = False
    ner_model.layers[-1].trainable = True
    ner_model.layers[-2].trainable = True
    
    return ner_model

model = bert_BiLSTM_CRF_model()

In [ ]:
total_steps, warmup_steps = calc_train_steps(
    num_example=data.shape[0],
    batch_size=batch_size,
    epochs=epochs,
    warmup_proportion=0.1,
)

optimizer = AdamWarmup(total_steps, warmup_steps, lr=1e-3, min_lr=1e-5)

callback_list = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
          #,ModelCheckpoint(filepath='AML_bert.h5', monitor='val_loss', save_best_only=True)
]

model.compile(optimizer=optimizer, loss=crf_loss, metrics=[crf_accuracy])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [ ]:
label1 = keras.utils.to_categorical(label, num_classes=3, dtype='float32')
label1[a][st:ed]

In [ ]:
model.fit([token_input, segment_input, mask_input], label1,
       epochs=epochs, batch_size=batch_size,
          #validation_split=0.1
          callbacks=callback_list)

Epoch 1/2
25000/25000 [==============================] - 1502s 60ms/step - loss: -0.0015 - crf_accuracy: 0.9999
Epoch 2/2
25000/25000 [==============================] - 1522s 61ms/step - loss: -0.0019 - crf_accuracy: 0.9999


In [ ]:
model.save(r'/gdrive/My Drive/law/law_ner.h5')

In [ ]:
model.load_weights(r'/gdrive/My Drive/law/law_ner.h5')

# 檢視train

In [ ]:
prediction = model.predict([token_input, segment_input, mask_input])
prediction[a][st:ed]

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1

In [ ]:
import numpy as np
y_pred = np.argmax(prediction, axis=-1)
# y_pred = y_pred-1

In [ ]:
print(len(label), len(label[0]))
print(y_pred.shape)
print(len(token_input), len(token_input[0]))

25000 512
(25000, 512)
25000 512


In [ ]:
print(label1[a][st:(ed-20)])
y_pred[a][st:ed]

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [ ]:
cates = y_pred[0]
tokens = token_input[0]
np.where(cates!=0)[0]
# token_input[0][404:407]
# token_dict_inv[token_input[0][404:407][0]]

array([198, 199, 200, 203, 204, 205, 208, 209, 210, 213, 214, 215])

In [ ]:
# token_id map回 token
token_dict_inv = {value: key for key, value in token_dict.items()}

In [ ]:
def get_name(token_input, y_pred):
    '拼湊pred的名稱'
    name_list = []
    # 每篇文
    for tokens, cates in zip(token_input, y_pred):
        # 找出分類不為0的
        temp_name_list= []
        name_index = np.where(cates!=0)[0]
        name = '' # 紀錄名字
        a = 0 # 紀錄是否連續
        for idx in name_index: # 不為0的位置
            # print(idx)
            if a==0 or idx==a+1: # a=0代表開頭, a==a+1代表連續位置
                name += token_dict_inv[tokens[idx]].replace('#', '') # 找出是什麼字
                a = idx
                if idx == name_index[-1]: # 最後位置
                    temp_name_list.append(name)
            else:
                temp_name_list.append(name)
                a = 0
                name = ''
                name += token_dict_inv[tokens[idx]].replace('#', '') # 找出是什麼字
        name_list.append(temp_name_list)
        
    return name_list
name_list = get_name(token_input, y_pred)

In [ ]:
data1 = data.copy()
data1['judge'] = judge
data1['pred'] = name_list
print(data1.iloc[0])

# 看看不一樣的地方
data2 = data1[data1['judge'] != data1['pred']]
print(data2.shape)

title                      司法院--刑事補償 109 年度 台覆 字第 101 號刑事決定書
url        data.aspx?ro=0&q=7a56f5a4cdd68d79ef3ef69427141...
content    審人即補償請求人劉傳文（下稱聲請人）\r\n因違反毒品危害防制條例案件，請求刑事補償，原決定...
judge                                   [徐昌錦, 鄭傑夫, 林恩山, 許錦印]
pred                                    [徐昌錦, 鄭傑夫, 林恩山, 許錦印]
Name: 0, dtype: object
(327, 5)


In [ ]:
# patt > model
patt_longer_model = data2.loc[data2['judge'].map(len) > data2['pred'].map(len)]
# patt < model
patt_shorter_model = data2.loc[data2['judge'].map(len) < data2['pred'].map(len)]
# patt == model，但不相同
patt_notequal_model = data2.loc[data2['judge'].map(len) == data2['pred'].map(len)]

print(patt_longer_model.shape, patt_shorter_model.shape, patt_notequal_model.shape)

(20, 5) (131, 5) (176, 5)


In [ ]:
# patt > model，模型未抓到
a = 3
print(patt_longer_model.iloc[a, [3, 4]])
patt_longer_model.iloc[a, 2][-510:]

judge    [陳文爵]
pred        []
Name: 644, dtype: object


'序後，得命司法事務官進行清算程序，消費\r\n者債務清理條例第16條第1項前段定有明定，爰併裁定如主\r\n文第3項。\r\n中華民國109年11月17日\r\n臺灣臺中地方法院民事庭\r\n法官陳文爵\r\n上為正本係照原本作成\r\n如不服本裁定關於生活限制部分，應於送達後10日內以書狀向本\r\n院提出抗告，並繳納抗告費新臺幣1千元；關於開始清算及命司\r\n法事務官進行清算程序部分，不得抗告。\r\n本裁定已於109年11月17日下午2時公告。\r\n中華民國109年11月17日\r\n\n附件：清算債務人之生活限制\r\n\r\n准許清算之債務人，在本件清算程序終止或終結前，應受下列之生活限制：\r\n\r\n一、不得為奢靡浪費之消費活動。\r\n\r\n二、不得為賭博或為其他投機行為。\r\n\r\n三、不得為不動產之處分。\r\n\r\n四、不得為金錢借貸之行為。\r\n\r\n五、不得搭乘計程車、高鐵及航空器，但債務人能提出確實證據證明因工作所需\r\n且未因此減少債權人受償金額者，不在此限。\r\n\r\n六、不得從事國外遊學或出國旅遊等消費行為。\r\n\r\n七、不得投資金融商品（例如股票、基金等）。\r\n\r\n八、不得從事逾越通常生活程度之贈與。\r\n\r\n九、其他經本院限制之行為。\r\n\r\n\n\n\n\n\n\n\n'

In [ ]:
# patt == model，長度一致卻不相同
a = 3
print(patt_notequal_model.iloc[a, [3, 4]])
patt_notequal_model.iloc[a, 2][-510:]

judge        [吳佳齡, 周霙蘭]
pred     [吳佳齡, 周[UNK]蘭]
Name: 501, dtype: object


'371號判例首揭判例意旨，自不發生\r\n回復原狀之問題，從而，聲請人聲請回復原狀亦屬無理由，\r\n應予駁回。\r\n六、另上開判決雖未合法送達，惟聲請人業於106年1月24日至七\r\n堵派出所領取判決，並於106年1月25日具狀提起上訴，有上\r\n訴狀首頁所載之本院收文戳章在卷可憑，已生合法上訴之效\r\n力，且有利於聲請人，本院茲因郵務機關漏未向本院陳明上\r\n情，致本院收受聲請人前開上訴案件時無從發見誤為逾期，\r\n而於106年2月10日從程序上為駁回上訴之裁定（見本院106\r\n年度聲字第25號卷第26-1頁），此種程序上裁定，不發生實\r\n質上確定力，聲請人之上訴不因而失效，自毋庸先依非常上\r\n訴程序撤銷（最高法院25年上字第3231號判例、同院80年11\r\n月5日第5次刑事庭會議決議參照），併予敘明。\r\n據上論斷，依刑事訴訟法第69條第1項前段，裁定如主文。\r\n中華民國10年6月8日\r\n刑事第五庭審判長法官齊潔\r\n法官吳佳齡\r\n法官周霙蘭\r\n以上正本證明與原本無異。\r\n對於本件裁定如有不服，應於收受送達後5日內，向本院提出抗\r\n告書狀，敘述抗告之理由，抗告於臺灣高等法院。\r\n中華民國106年6月8日\r\n\n\n\n\n\n\n\n\n'

In [ ]:
# patt < model，規則未抓到
a = 3
print(patt_shorter_model.iloc[a, [3, 4]])
patt_shorter_model.iloc[a, 2][-510:]

judge       []
pred     [林柏泓]
Name: 81, dtype: object


'為最輕本刑5年以上有期徒刑之罪，基於一般人畏懼長期刑執行之心理，尚難排除為免執行而逃亡之可能，是前項羈押原因依然存在。再審酌羈押之目的在保全刑事審判及執行之進行，並確保刑事審判機關得以依法從事犯罪事實之調查與認定，及擔保執行之必要性，本案現尚未確定，仍有保全審判進行及擔保未來確定後執行之必要性。至於被告雖辯稱其罹患痛風，長期服藥，且名下財產均遭扣押，並無逃亡之虞云云。然查，被告之疾病僅屬慢性病症，不易造成生命危害，亦得服藥控制疼痛，且被告本案遭扣押之物均係名義上屬其所有之財產，但仍不能排除接受親友援助或以其他方式取得接濟之可能，是上開理由，均無解於延長其羈押期間之必要性。經權衡國家刑事司法權之有效行使、被告人身自由之私益及防禦權受限制之程度後，認依目前訴訟進行程度，尚不能以具保等處分代替羈押，而有繼續羈押之必要。三、綜上，本件羈押原因尚未消滅，尚有羈押被告之原因及必要，爰裁定如主文。據上論斷，依刑事訴訟法第108條第1項、第5項，裁定如主文。中華民國109年11月16日刑事第五庭審判長法官王屏夏法官戴嘉清法官林柏泓以上正本證明與原本無異。如不服本裁定，應於收受送達後五日內向本院提出抗告狀。\n\n\n\n\n\n\n\n'

# test

In [ ]:
import numpy as np

# token_id map回 token
token_dict_inv = {value: key for key, value in token_dict.items()}

def get_name(token_input, y_pred):
    '拼湊pred的名稱'
    name_list = []
    # 每篇文
    for tokens, cates in zip(token_input, y_pred):
        # 找出分類不為0的
        temp_name_list= []
        name_index = np.where(cates!=0)[0]
        name = '' # 紀錄名字
        a = 0 # 紀錄是否連續
        for idx in name_index: # 不為0的位置
            # print(idx)
            if a==0 or idx==a+1: # a=0代表開頭, a==a+1代表連續位置
                name += token_dict_inv[tokens[idx]].replace('#', '') # 找出是什麼字
                a = idx
                if idx == name_index[-1]: # 最後位置
                    temp_name_list.append(name)
            else:
                temp_name_list.append(name)
                a = 0
                name = ''
                name += token_dict_inv[tokens[idx]].replace('#', '') # 找出是什麼字
        name_list.append(temp_name_list)
        
    return name_list


In [ ]:
token_test, segment_test, mask_test = encoded(tokenizer, test, maxlen)

t = time.time()
prediction_test = model.predict([token_test, segment_test, mask_test])
time.time() - t

98.10316681861877

In [ ]:
judge_test = test['content'].str[-510:].str.findall(r'\n法官(.{2,5})\r')
label_test = create_label_crf(tokenizer, token_test, judge_test, maxlen)

In [ ]:
b = 6
idx_test = label_test[b].index(1)
st_test = idx_test-10
ed_test = idx_test+10
token_test[b][st_test:ed_test]

[2832,
 1765,
 3175,
 3791,
 7368,
 3696,
 752,
 2431,
 3791,
 2135,
 7376,
 2134,
 6545,
 1381,
 4158,
 3633,
 3315,
 913,
 4212,
 1333]

In [ ]:
print(test.iloc[b, 2][st_test:ed_test])
print([token_dict_inv[tt] for tt in token_test[b][st_test:ed_test]])
print(token_test[b][st_test:ed_test])
print(label_test[b][st_test:ed_test])
print(y_pred_test[b][st_test:ed_test])
name_list_test[b]

十九年二月十五日
臺灣南投地方法院民事
['投', '地', '方', '法', '院', '民', '事', '庭', '法', '官', '陳', '宗', '賢', '右', '為', '正', '本', '係', '照', '原']
[2832, 1765, 3175, 3791, 7368, 3696, 752, 2431, 3791, 2135, 7376, 2134, 6545, 1381, 4158, 3633, 3315, 913, 4212, 1333]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0]
[ 0  0  0  0  0  0  0  0  0  0 -1 -1 -1  0  0  0  0  0  0  0]


['條第2']

In [ ]:
y_pred_test = np.argmax(prediction_test, axis=-1)
y_pred_test = y_pred_test-1
name_list_test = get_name(token_test, y_pred_test)


test1 = test.copy()
test1['judge'] = judge_test
test1['pred'] = name_list_test
print(test1.iloc[0])
print(test1.shape)

title                           臺灣南投地方法院 89 年度 訴 字第 94 號民事判決
url        data.aspx?ro=440&q=7a56f5a4cdd68d79ef3ef694271...
content    確認印鑑證明無效等\r\n\n\n\n\n臺灣南投地方法院民事判決八十九年度訴字第九四號\r...
judge                                                     []
pred                                                      []
Name: 25000, dtype: object
(1471, 5)


In [ ]:
# 看看不一樣的地方
test2 = test1.loc[test1['judge'] != test1['pred']]

# 一樣都有 = True Positive
print(sum((test1['judge'] == test1['pred']) & (test1['judge'].map(len) > 0)))

# 一樣都沒有 = True Nagative
print(sum((test1['judge'] == test1['pred']) & (test1['judge'].map(len) == 0)))

# 不一樣，真實多於預測 = False Nagative
print(sum(test1['judge'].map(len) > test1['pred'].map(len)))

# 不一樣，預測多於真實 = False Positive
print(sum(test1['judge'].map(len) < test1['pred'].map(len)))

# 數量一樣，但內容不完全一樣
test2.loc[test2['judge'].map(len) == test2['pred'].map(len)].shape

261
1192
5
8


(5, 5)

In [ ]:
# precision
print(f'precision: {261/(261+8)}')
# recall
print(f'recall: {261/(261+5)}')
# f1_score
print(f'f1_score: {2*(0.97*0.98)/(0.97+0.98)}')

precision: 0.9702602230483272
recall: 0.981203007518797
f1_score: 0.974974358974359


In [ ]:
# patt > model
patt_longer_model = test2.loc[test2['judge'].map(len) > test2['pred'].map(len)]
# patt < model
patt_shorter_model = test2.loc[test2['judge'].map(len) < test2['pred'].map(len)]
# patt == model，但不相同
patt_notequal_model = test2.loc[test2['judge'].map(len) == test2['pred'].map(len)]

print(patt_longer_model.shape, patt_shorter_model.shape, patt_notequal_model.shape)

(5, 5) (8, 5) (5, 5)


In [ ]:
patt_longer_model.iloc[:, 2:5]

,content,judge,pred
25302,0元」，見警\r\n卷第23頁），但告訴人表示尚未修繕，復因另行租車使用，\r\n每個月要4...,[粘凱庭],[]
25570,本院107年度上\r\n字第197號）之上訴人，為依法得聲請閱覽卷宗之人，復據\r\n聲請人...,"[黃義成, 藍雅清]",[]
25584,"0,500元，有稅務電子閘門財產調件明\r\n細表在卷（見原審補字卷第63頁）可查，顯高於債...","[陳春長, 林富郎]",[陳春長]
25585,"應徵裁判費。查本件訴訟標的金額為新臺幣\r\n（下同）141,887元，應徵裁判費2,325...","[王浦傑, 張季芬]",[]
25709,\r\n法官劉台安\r\n法官賴劍毅\r\n右為正本係照原本作成\r\n不得上訴\r\n中華...,"[劉台安, 賴劍毅]",[賴劍毅]


In [ ]:
patt_shorter_model.iloc[:, 2:5]

,content,judge,pred
25165,法官陳嘉惠\r\n右正本證明與原本無異\r\n中華民國88年5月31日\r\n法院\n上抄本...,[],[陳嘉惠]
25461,情形，應於提起上訴或委任時釋明之。上訴人未依第1項、第2項規定委任訴訟代理人，或雖依第2項委...,[],[羅立德]
25462,"6,002元。茲命上訴人於收受本裁定正本送達後5日內補正委任律師或具律師資格關係人之委任狀及...",[],[羅立德]
25463,"萬5,655元。茲命上訴人於收受本裁定正本之日起5日內，如數向本院補繳，並提出委任律師或具律...",[],[何君豪]
25466,109年12月24日上午9時30分在本院第12法庭行準備程序，特此裁定。中華民國109年11...,[],[許勻睿]
25473,律師為訴訟代理人。但上訴人或其法定代理人具有律師資格者，不在此限。上訴人之配偶、三親等內之血...,[],"[朱美[UNK], 何君豪]"
25719,庭法官黃書苑\r\n右正本證明與原本無異\r\n本裁定不得抗告\r\n中華民國八十九年一月五...,[],[黃書苑]
25876,B法官黃麟倫\r\n右正本證明與原本無異。\r\n對於本件判決如有不服，應於收受送達後二十日...,[],[黃麟倫]


In [ ]:
patt_notequal_model.iloc[:, 2:5].head(10)

,content,judge,pred
25301,附帶民事訴訟裁定\r\n109年度簡上附民字第15號\r\n附民原告鄧玉恆\r\n附民被告劉...,"[粘凱庭, 沈婷勻]","[粘凱, 沈婷勻]"
25476,花\r\n訴訟代理人楊金順律師\r\n複代理人吳奎新律師\r\n被上訴人\r\n即上訴人正昇...,"[蔡烱燉, 高孟焄]","[蔡[UNK]燉, 高孟[UNK]]"
25492,前段分別定有明文。則依此規定，被上訴人羅玉惠、黃天保、許見\r\n清、彭武彬分別請求上訴人給...,"[蔡烱燉, 王聖惠]","[蔡[UNK]燉, 王聖惠]"
25493,事實，因之系爭土地既業經重慶國小種植樹木\r\n使用中，本件房屋已無在原址重建回復原狀之可能...,"[王聖惠, 蔡烱燉]","[王聖惠, 蔡[UNK]燉]"
25713,北地方法院交通法庭\r\n法官鍾華\r\n右正本證明與原本無異。\r\n如不服本裁定，應於裁...,[鍾華],[鍾華右]
